<a href="https://colab.research.google.com/github/edanielacero/SIS-INT-2023-PRAC-3/blob/main/Proyecto_oficial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import random

In [1]:
class Estado:
    def __init__(self, tamano=8):
        self.tamano = tamano
        self.tablero = [['.' for _ in range(tamano)] for _ in range(tamano)]
        # Inicializar fichas
        medio = tamano // 2
        self.tablero[medio-1][medio-1] = self.tablero[medio][medio] = 'O'
        self.tablero[medio][medio-1] = self.tablero[medio-1][medio] = 'X'

        self.letras_columna = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'[:tamano]


    def mostrar_tablero(self):
        print('  ' + ' '.join(self.letras_columna))

        for i, fila in enumerate(self.tablero):
            print(f'{i+1} ', end='')

            for casilla in fila:
                print(casilla, end=' ')
            print()
        print()

    def letra_a_numero_columna(self, letra):
        return ord(letra) - 65  # Convierte la letra a un número de columna (0-7)

    def realizar_movimiento(self, fila, columna_letra, jugador):
      fila = fila - 1
      columna = self.letra_a_numero_columna(columna_letra)
      if self.tablero[fila][columna] == ".":
        self.tablero[fila][columna] = jugador
        adversario = 'O' if jugador == 'X' else 'X'
        direcciones = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, 1), (1, -1), (-1, -1)]

        for dir_f, dir_c in direcciones:
            i, j = fila + dir_f, columna + dir_c
            if not self.esta_dentro_del_tablero(i, j) or self.tablero[i][j] != adversario:
                continue

            i, j = i + dir_f, j + dir_c
            while self.esta_dentro_del_tablero(i, j) and self.tablero[i][j] == adversario:
                i, j = i + dir_f, j + dir_c

            if self.esta_dentro_del_tablero(i, j) and self.tablero[i][j] == jugador:
                while (i != fila or j != columna):
                    i, j = i - dir_f, j - dir_c
                    self.tablero[i][j] = jugador

    def obtener_movimientos_validos(self, jugador):
        movimientos_validos = []

        for i in range(self.tamano):
            for j in range(self.tamano):
                if self.tablero[i][j] == '.':
                    if self.es_movimiento_valido(i, j, jugador):
                        fila_numero = i + 1
                        columna_letra = chr(j + 65)  # Convertimos el índice a letra (A-H)
                        movimientos_validos.append((fila_numero, columna_letra))

        return movimientos_validos

    def es_movimiento_valido(self, fila, columna, jugador):
        if self.tablero[fila][columna] != '.':
            return False

        adversario = 'O' if jugador == 'X' else 'X'
        direcciones = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, 1), (1, -1), (-1, -1)]

        for dir_f, dir_c in direcciones:
            i, j = fila + dir_f, columna + dir_c
            if self.esta_dentro_del_tablero(i, j) and self.tablero[i][j] == adversario:
                i, j = i + dir_f, j + dir_c
                while self.esta_dentro_del_tablero(i, j) and self.tablero[i][j] == adversario:
                    i, j = i + dir_f, j + dir_c

                if self.esta_dentro_del_tablero(i, j) and self.tablero[i][j] == jugador:
                    return True

        return False

    def esta_dentro_del_tablero(self, fila, columna):
        return 0 <= fila < self.tamano and 0 <= columna < self.tamano

    def es_juego_terminado(self):
       return self.tablero_lleno()

    def tablero_lleno(self):
      cont = sum(1 for fila in self.tablero for casilla in fila if casilla != ".")
      return cont == self.tamano * self.tamano

    def obtener_ganador(self):
        contador_X = sum(fila.count('X') for fila in self.tablero)
        contador_O = sum(fila.count('O') for fila in self.tablero)

        if contador_X > contador_O:
            return 'X'
        elif contador_O > contador_X:
            return 'O'
        else:
            return 'Empate'

    def utility(self):
      contador_X = sum(fila.count('X') for fila in self.tablero)
      contador_O = sum(fila.count('O') for fila in self.tablero)
      return contador_X - contador_O

In [2]:
est = Estado()
est.mostrar_tablero()
print(est.obtener_movimientos_validos('X'))
est.realizar_movimiento(3, 'D', 'X')
est.mostrar_tablero()
print(est.es_juego_terminado())
est.utility()

  A B C D E F G H
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . . . . . . 
4 . . . O X . . . 
5 . . . X O . . . 
6 . . . . . . . . 
7 . . . . . . . . 
8 . . . . . . . . 

[(3, 'D'), (4, 'C'), (5, 'F'), (6, 'E')]
  A B C D E F G H
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . X . . . . 
4 . . . X X . . . 
5 . . . X O . . . 
6 . . . . . . . . 
7 . . . . . . . . 
8 . . . . . . . . 

False


3

In [3]:
est = Estado(4)
est.mostrar_tablero()

  A B C D
1 . . . . 
2 . O X . 
3 . X O . 
4 . . . . 



In [4]:
import copy

## Min max

In [5]:
def terminal_test(state):
  return state.es_juego_terminado()

In [6]:
def result(state, action):
  succesor = copy.deepcopy(state)
  succesor.realizar_movimiento(action[0][0], action[0][1], action[1])
  #succesor.mostrar_tablero()
  return succesor

In [7]:
def utility(state):
  return state.utility()

In [8]:
def max_value(state):
    if terminal_test(state):
        return utility(state)

    v = float('-inf') # valor pequeño
    actions = state.obtener_movimientos_validos('X')
    print(actions)
    for action in actions:
        print("Estoy en las acciones de max")
        successor = result(state, (action,'X')) # devuelve estado sucesor
        eval_min = min_value(successor) # evaluar el estado
        v = max(v, eval_min)
    return v

def min_value(state):
    if terminal_test(state):
        return utility(state)

    v = float('inf') # valor grande
    actions = state.obtener_movimientos_validos('O')
    print(actions)
    for action in actions:
        print("Estoy en las acciones de min")
        successor = result(state, (action,'O')) # devuelve estado sucesor
        eval_max = max_value(successor) # evaluar el estado
        v = min(v, eval_max)
    return v

def min_max(state, player):
    if player == "MAX":
        v = []
        actions = state.obtener_movimientos_validos('X')
        print()
        for action in actions:
            print("En mix max superior")
            successor = result(state, (action,'X'))
            v.append((min_value(successor), action))
        return max(v, key=lambda x: x[0])[1]

    elif player == "MIN":
        v = []
        actions = state.obtener_movimientos_validos('O')
        for action in actions:
            successor = result(state, (action,'O'))
            v.append((max_value(successor), action))
        return min(v, key=lambda x: x[0])[1]

In [9]:
initial_state = Estado(4)
min_max(initial_state, "MAX")

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Estoy en las acciones de max
[(4, 'A')]
Estoy en las acciones de min
Estoy en las acciones de min
[(1, 'D'), (3, 'A'), (4, 'A'), (4, 'D')]
Estoy en las acciones de max
[]
Estoy en las acciones de max
[(4, 'D')]
Estoy en las acciones de min
[]
Estoy en las acciones de max
[(3, 'A'), (4, 'D')]
Estoy en las acciones de min
[(1, 'D'), (4, 'D')]
Estoy en las acciones de max
[]
Estoy en las acciones de max
[(1, 'D')]
Estoy en las acciones de min
Estoy en las acciones de min
[(3, 'A')]
Estoy en las acciones de max
[]
Estoy en las acciones de max
[(1, 'D')]
Estoy en las acciones de min
[(3, 'A'), (4, 'A')]
Estoy en las acciones de max
[(4, 'A')]
Estoy en las acciones de min
Estoy en las acciones de max
[]
Estoy en las acciones de min
[(3, 'A'), (4, 'A'), (4, 'B')]
Estoy en las acciones de max
[(4, 'B')]
Estoy en las acciones de min
[]
Estoy en las acciones de max
[(3, 'A'), (4, 'B')]
Estoy en las acciones de min
[(4, 'B')]
Esto

KeyboardInterrupt: ignored

# Min Max 𝞪-𝜷 pruning

In [16]:
def max_value(state, alpha, beta):
    if terminal_test(state):
        return utility(state)

    v = float('-inf') # valor pequeño
    actions = state.obtener_movimientos_validos('X')
    #print(actions)
    random.shuffle(actions)
    for action in actions:
        if(alpha>=beta):
          continue
        #print("Estoy en las acciones de max")
        successor = result(state, (action,'X')) # devuelve estado sucesor
        eval_min = min_value(successor, alpha, beta) # evaluar el estado
        if eval_min > v:
          v = eval_min
          if v > alpha:
            alpha = v
    return v

def min_value(state, alpha, beta):
    if terminal_test(state):
        return utility(state)

    v = float('inf') # valor grande
    actions = state.obtener_movimientos_validos('O')
    #print(actions)
    random.shuffle(actions)
    for action in actions:
        if alpha >= beta:
          continue
        #print("Estoy en las acciones de min")
        successor = result(state, (action,'O')) # devuelve estado sucesor
        eval_max = max_value(successor, alpha, beta) # evaluar el estado
        if eval_max < v:
          v = eval_max
          if beta > v:
            beta = v
    return v

def min_max_pruning(state, player):
    alpha = float('-inf') # valor pequeño
    beta = float('inf') # valor grande
    if player == "MAX":
        v = []
        actions = state.obtener_movimientos_validos('X')
        for action in actions:
            #print("En mix max superior")
            successor = result(state, (action,'X'))
            v.append((min_value(successor, alpha, beta), action))
        return max(v, key=lambda x: x[0])[1]

    elif player == "MIN":
        v = []
        actions = state.obtener_movimientos_validos('O')
        for action in actions:
            successor = result(state, (action,'O'))
            v.append((max_value(successor,alpha, beta), action))
        return min(v, key=lambda x: x[0])[1]

In [11]:
initial_state = Estado(4)
min_max_pruning(initial_state, "MAX")

(1, 'B')

In [12]:
def jugador_vs_maquina(tamano_tablero):
    estado = Estado(tamano_tablero)
    turno = 'O'  #  comienza jugando

    while not estado.es_juego_terminado():
        estado.mostrar_tablero()

        if turno == 'X':  # Turno del jugador
            print("Es tu turno.")
            movimientos_validos = estado.obtener_movimientos_validos('X')
            print("Movimientos válidos:")
            for i, (fila, columna) in enumerate(movimientos_validos):
                print(f"{i+1}. Fila: {fila}, Columna: {columna}")

            seleccion = int(input("Elige un movimiento (número): ")) - 1
            fila, columna = movimientos_validos[seleccion]
            estado.realizar_movimiento(fila, columna, 'X')

        else:  # Turno de la máquina (IA)
            print("Turno de la máquina (IA).")
            estado_copia = copy.deepcopy(estado)
            jugada = min_max_pruning(estado_copia, 'MAX')
            estado.realizar_movimiento(jugada[0], jugada[1], 'O')
            print("La computadora elijio: ", jugada)
        turno = 'O' if turno == 'X' else 'X'

    estado.mostrar_tablero()
    ganador = estado.obtener_ganador()

    if ganador == 'Empate':
        print("¡Es un empate!")
    else:
        print(f"¡El ganador es el jugador {ganador}!")

In [19]:
jugador_vs_maquina(4)

  A B C D
1 . . . . 
2 . O X . 
3 . X O . 
4 . . . . 

Turno de la máquina (IA).
La computadora elijio:  (1, 'B')
  A B C D
1 . O . . 
2 . O X . 
3 . X O . 
4 . . . . 

Es tu turno.
Movimientos válidos:
1. Fila: 2, Columna: A
2. Fila: 3, Columna: D
3. Fila: 4, Columna: C
Elige un movimiento (número): 1
  A B C D
1 . O . . 
2 X X X . 
3 . X O . 
4 . . . . 

Turno de la máquina (IA).
La computadora elijio:  (3, 'D')
  A B C D
1 . O . . 
2 X X O . 
3 . X O O 
4 . . . . 

Es tu turno.
Movimientos válidos:
1. Fila: 1, Columna: D
2. Fila: 2, Columna: D
3. Fila: 4, Columna: D
Elige un movimiento (número): 2
  A B C D
1 . O . . 
2 X X X X 
3 . X O O 
4 . . . . 

Turno de la máquina (IA).
La computadora elijio:  (4, 'D')
  A B C D
1 . O . . 
2 X X X X 
3 . X O O 
4 . . . O 

Es tu turno.
Movimientos válidos:
1. Fila: 4, Columna: B
2. Fila: 4, Columna: C
Elige un movimiento (número): 2
  A B C D
1 . O . . 
2 X X X X 
3 . X X O 
4 . . X O 

Turno de la máquina (IA).


ValueError: ignored